In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Polygon, Point
import numpy as np

from typing import List

from asim.dataset.dataset_specific.carla.opendrive.elements.lane import Lane, LaneSection
from asim.dataset.dataset_specific.carla.opendrive.elements.reference import Border

In [ ]:
from pathlib import Path
from asim.dataset.dataset_specific.carla.opendrive.elements.opendrive import OpenDrive

town_name = "Town03"

if town_name not in ["Town11", "Town12", "Town13", "Town15"]:
    carla_maps_root = Path("/home/daniel/carla_workspace/carla_garage/carla/CarlaUE4/Content/Carla/Maps/OpenDrive")
    carla_map_path = carla_maps_root / f"{town_name}.xodr"

else:
    carla_map_path = f"/home/daniel/carla_workspace/carla_garage/carla/CarlaUE4/Content/Carla/Maps/{town_name}/OpenDrive/{town_name}.xodr"

opendrive = OpenDrive.parse_from_file(carla_map_path)

road_dict = {road.id: road for road in opendrive.roads}

In [ ]:
from asim.dataset.dataset_specific.carla.opendrive.opendrive_converter import OpenDriveConverter


converter =  OpenDriveConverter(opendrive)

converter.run()

In [ ]:


center_polyline_3 = converter.lane_helper_dict["41_0_right_-1"].center_polyline_3d


plt.plot(center_polyline_3[...,0], center_polyline_3[..., 2])


In [ ]:
# road_dict = {road.id: road for road in opendrive.roads}


# fix, ax = plt.subplots(1,2)


# road = road_dict[7]

# elevations = road.elevation_profile.elevations
# plan_view = road.plan_view

# s_values = np.linspace(0, plan_view.length, 100, endpoint=True)


# xyz = np.array([plan_view.interpolate_xyz(s, elevations) for s in s_values])

# # xyz[...,2] = np.cumsum(xyz[...,2])


# ax[0].plot(xyz[...,0], xyz[...,2])
# ax[1].plot(xyz[...,1], xyz[...,2])

# for ax_ in ax:

#     ax_.set_aspect("equal")


In [ ]:
import numpy as np
import trimesh

def create_lane_mesh(left_boundary, right_boundary):
    """
    Create a 3D mesh from left and right boundary polylines.
    
    Parameters:
    - left_boundary: numpy array with shape (n, 3) representing left lane boundary
    - right_boundary: numpy array with shape (n, 3) representing right lane boundary
    
    Returns:
    - trimesh.Trimesh object representing the lane as a 3D mesh
    """
    # Ensure both polylines have the same number of points
    if left_boundary.shape[0] != right_boundary.shape[0]:
        raise ValueError("Both polylines must have the same number of points")
    
    n_points = left_boundary.shape[0]
    
    # Combine vertices from both polylines
    vertices = np.vstack([left_boundary, right_boundary])
    
    # Create faces by connecting corresponding points on the two polylines
    faces = []
    for i in range(n_points - 1):
        # Add two triangles for each quad
        # Triangle 1: (left_i, right_i, left_i+1)
        faces.append([i, i + n_points, i + 1])
        # Triangle 2: (left_i+1, right_i, right_i+1)
        faces.append([i + 1, i + n_points, i + n_points + 1])
    
    faces = np.array(faces)
    
    # Create the mesh
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    return mesh

In [ ]:
import time
from pathlib import Path

import numpy as np
import trimesh

import viser
import viser.transforms as tf


# lane_id = "494_0_right_-1" 

server = viser.ViserServer()

for lane_id in converter.lane_helper_dict.keys():
    if converter.lane_helper_dict[lane_id].type != "driving":
        continue


    left_boundary = converter.lane_helper_dict[lane_id].inner_polyline_3d
    right_boundary = converter.lane_helper_dict[lane_id].outer_polyline_3d
    centerline = converter.lane_helper_dict[lane_id].center_polyline_3d

    mesh = create_lane_mesh(left_boundary, right_boundary)
    # mesh.show()

    
    # server.scene.add_mesh_simple(
    #     name="/simple",
    #     vertices=mesh.vertices,
    #     faces=mesh.faces,
    #     wxyz=tf.SO3.from_x_radians(np.pi / 2).wxyz,
    #     position=(0.0, 0.0, 0.0),
    # )
    server.scene.add_mesh_trimesh(
        name=f"/trimesh/{lane_id}",
        mesh=mesh.smoothed(),
        # wxyz=tf.SO3.from_x_radians(np.pi / 2).wxyz,
        position=(0.0, 0.0, 0.0),
    )
    centerline_points = np.concatenate([centerline[:-1, None], centerline[1:, None]], axis=1)

    centerline_points[..., 2] += 0.1
    # print(centerline_points.shape)

    server.scene.add_line_segments(
        f"/line_segments/{lane_id}",
        points=centerline_points,
        colors="black",
        line_width=3.0,
    )

while True:
    time.sleep(10.0)


In [ ]:
for road in road_dict.values():

    # super_elevations = road.lateral_profile.super_elevations
    # polynoms = road.lateral_profile.shapes
    polynoms = road.elevation_profile.elevations
    # polynoms = road.lateral_profile.super_elevations
    # coeffs = np.array([polynom.polynomial_coefficients for polynom in polynoms])
    coeffs = np.array([polynom.polynomial_coefficients for polynom in polynoms])

    if np.any(coeffs != 0):
        print(road.name)




In [ ]:
# types = []
# for lane_helper in converter.lane_helper_dict.values():
#     types += [lane_helper.type]


# types = print(town_name, set(types))
# types

# Town03 {'driving', 'median', 'none', 'bidirectional', 'parking', 'sidewalk', 'border', 'shoulder'}


# # {'driving', 'shoulder', 'sidewalk'}
# # {"bidirectional", "border", "driving", "median", "none", "parking", "shoulder", "sidewalk"}
# # {'driving', 'none', 'shoulder', 'sidewalk'}2
# # {'driving', 'parking', 'shoulder', 'sidewalk'}
# # {'driving', 'median', 'none', 'shoulder'}

gdf = converter._extract_lane_dataframe()
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax)

fig.savefig("town_3.pdf")


In [ ]:
gdf = converter._extract_lane_group_dataframe()
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax)

fig.savefig("town_3.pdf")

In [ ]:

gdf = converter._extract_walkways_dataframe()

gdf.plot()

In [ ]:

gdf = converter._extract_carpark_dataframe()

gdf.plot()

In [ ]:
len(converter.lane_helper_dict)

In [ ]:
from asim.common.visualization.color.lib.opendrive import OPENDRIVE_LANE_CUSTOM


def plot_opendrive_converter(converter: OpenDriveConverter) -> None:

    fig, ax = plt.subplots(figsize=(50, 50))

    for lane_helper in converter.lane_helper_dict.values():
        polygon = lane_helper.shapely_polygon
        exterior_x, exterior_y = polygon.exterior.xy
        ax.fill(
            exterior_x,
            exterior_y,
            color=OPENDRIVE_LANE_CUSTOM[lane_helper.type].hex,
            linewidth=0.0,
            alpha=1.0,
        )
        # if lane_helper.type == "driving":
        #     centerline = lane_helper.center_polyline

        #     ax.plot(
        #         centerline[..., 0],
        #         centerline[..., 1],
        #         color="black",
        #         linewidth=0.1,
        #         alpha=0.5,
        #     )



    ax.set_aspect("equal")
    return fig, ax

fig, ax = plot_opendrive_converter(converter)
fig.savefig("test.pdf")

In [ ]:
from asim.common.visualization.color.lib.opendrive import OPENDRIVE_LANE_CUSTOM


def plot_opendrive_converter_lane_groups(converter: OpenDriveConverter) -> None:

    fig, ax = plt.subplots(figsize=(10, 10))

    for lane_group_helper in converter.lane_group_helper_dict.values():
        polygon = lane_group_helper.shapely_polygon
        exterior_x, exterior_y = polygon.exterior.xy
        ax.fill(
            exterior_x,
            exterior_y,
            linewidth=0.0,
            alpha=1.0,
        )
        # color=OPENDRIVE_LANE_CUSTOM[lane_helper.type].hex,
        # if lane_helper.type == "driving":
        #     centerline = lane_helper.center_polyline

        #     ax.plot(
        #         centerline[..., 0],
        #         centerline[..., 1],
        #         color="black",
        #         linewidth=0.1,
        #         alpha=0.5,
        #     )



    ax.set_aspect("equal")
    return fig, ax

fig, ax = plot_opendrive_converter_lane_groups(converter)
fig.savefig("test.pdf")